In [ ]:
!nvidia-smi

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

In [ ]:
config = ConfigProto()

config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
# IMPORTING THE NECESSARY LAYERS

import numpy as np

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten

from keras.applications.vgg16 import VGG16

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential

from glob import glob


In [ ]:
IMAGE_SIZE = [224, 224]

train_path = "drive/MyDrive/DL/Krish Naik/train"
valid_path = "drive/MyDrive/DL/Krish Naik/test"


In [ ]:
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights="imagenet", include_top=False)

In [ ]:
for layer in vgg16.layers:
  layer.trainable = False

In [ ]:
folders = glob("train/*")

In [ ]:
folders

In [ ]:
len(folders)

In [ ]:
x = Flatten()(vgg16.output)

In [ ]:
preds = Dense(len(folders), activation="softmax")(x)

model = Model(inputs=vgg16.input, outputs=preds)

In [ ]:
model.summary()

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory('location',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

In [ ]:

test_set = test_datagen.flow_from_directory('location',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

In [ ]:
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

In [ ]:
from tensorflow.keras.models import load_model

model.save('model_vgg16.h5')